In [2]:
from IPython.display import display, Image, clear_output, HTML
import time
import ipywidgets as widgets
import json 
import random
from jupyter_ui_poll import ui_events
import requests
from bs4 import BeautifulSoup
import pandas as pd

#List of filenames
all_images = ["Variation1.jpg","Variation2.jpg","Variation3.jpg","Variation4.jpg","Variation5.jpg",
              "Variation6.jpg","Variation7.jpg","Variation8.jpg"]


def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

#Google form url
form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSeh-CmaGT4N6TKWmhvyhYMsmgtjHZ15sV7s2o_8_G_d7ouYwg/viewform'

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()
    """
    set event info to be empty
    as this is dict we can change entries
    directly without using
    the global keyword
    """
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    """
    Display button description in output area
    Button is case insensitive
    If the description of the clicked button matches the correct answer, correct_answer increases by 1 and right_answer equals 1
    If the description of the clicked button doesn't match the correct answer, right answer equals 0
    right_answer variable allows us to know whether the user gets each question right or wrong
    """
    global correct_answers, right_answer
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    if btn.description.lower() == correct_side:  
        correct_answers += 1
        right_answer = 1
    else:
        right_answer = 0

# Make the test the same for all users
random.seed(1)

print("Welcome to the ANS test")

# Inserting user ID
id_instructions = """

Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor / actress

e.g. if your friend was called Charlie Brown and film star was Tom Cruise
     then your unique identifier would be CBTC
"""

print(id_instructions)
# Store user input in variable 'ans1'
ans1 = input("> ")

# Clear output
clear_output(wait=False)

# Data consent form
data_consent_info = """DATA CONSENT INFORMATION:

Please read:

We wish to record your response data
to an anonymised public data repository. 
Your data will be used for educational teaching purposes
practising data analysis and visualisation.

Please type   yes   in the box below if you consent to the upload."""

print(data_consent_info)
# Store user input in variable 'rslt'
rslt = input("> ") 

# Check if variable 'rslt' equals the string "yes"
if rslt == "yes": 
    print("Thanks for your participation.")
    print("Please contact a.fedorec@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
    time.sleep(2)
else: 
    # Stop the test if 'rslt' does not equal "yes"
    raise(Exception("User did not consent to continue test."))

# Cleat output
clear_output(wait=False)

print("Please enter your gender (male,female or other):")
# Store user input in variable 'ans2'
ans2 = input("> ")

print("How many hours of sleep did you get last night?")
# Store user input in variable 'ans3'
ans3 = input(">")

# Clear output
clear_output(wait=False)

# Create an HTML widget to display instructions
instructions_html = widgets.HTML(value="""
<h3>Approximate Number System test</h3>
<p>Select either the left or right circle to indicate which one you believe contains the most dots</p>
<p>You will be shown the image for 0.75 seconds and have a response time of 3 seconds.</p>
<p>There will be 50 questions.</p>
""")
# Show instrutions for 20 seconds
display(instructions_html)
time.sleep(15)
clear_output(wait=False)

# Show for 5 seconds
print("The test will start in 5 seconds")
time.sleep(5)
clear_output(wait=False)

# Set number of questions 
questions = 50
# Start correct answers at 0 
correct_answers = 0

# Intialize dictionary
# Create empty list to store data for each key
results_dict = {
    'filename' : [],
    'rsponse_time' : [],
    'ratio' : [],
    'correct' : []
}

# Initializa for loop that iterates through number of questions
for i in range(questions):

    # Print question number
    display(HTML(f"<h2>Question: {i+1}</h2>"))
    
    # Randomize order of all_images
    random_image = random.choice(all_images)
    # Display image. Set the width
    display(Image(filename=random_image, width =700))

    # Show for only 0.75 seconds
    time.sleep(0.75)
    clear_output(wait=False)

    # Print question number again after clear_output
    display(HTML(f"<h2>Question: {i+1}</h2>"))
    
    # Record the current time and assign it to the variable `start_time`.
    start_time = time.time() 

    # Initialize two button widgets for each option
    # increase font size of description
    btn1 = widgets.Button(description="Left",style=dict(font_size="18px"))
    btn2 = widgets.Button(description="Right",style=dict(font_size="18px"))

    # Register `register_event` function as an event handler for the buttons.
    btn1.on_click(register_event) 
    btn2.on_click(register_event) 

    # Set width and height for each button 
    btn1.layout.width = '300px'  
    btn1.layout.height = '100px' 
    
    btn2.layout.width = '300px' 
    btn2.layout.height = '100px'  

    # Display btn1 and btn2 horizontally
    panel = widgets.HBox([btn1, btn2])
    display(panel)

    # Check the numeric value of the fifth character from the end of the string `random_image'
    if int(random_image[-5]) in [1, 4, 6, 7]:
        # If the numeric value is 1, 4, 6, 7, the variable `correct_side` is set to the string "right".
        correct_side = "right"
    else:
        # If not, the variable `correct_side` is set to the string "left".
        correct_side = "left"

    # Assign a ratio value based on the fifth character from the end of the string `random_image'
    # Ratio values are calculated from the images
    if int(random_image[-5]) in [1,2,3]: 
        rtio = 0.75
    elif int(random_image[-5]) in [4,5]:
        rtio = 0.857
    elif int(random_image[-5]) in [6]:
        rtio = 0.889
    else:
        rtio = 0.9
        
    # Set timeout to 3 seconds before buttons are cleared    
    result = wait_for_event(timeout=3)
    clear_output()
    
    # Record the current time and assign it to `end_time`
    end_time = time.time()
    # Calculate the response time by subtracting `start_time` from `end_time`.
    response_time =  end_time - start_time

    # Check if the 'description' key in the 'result' dictionary is not an empty string.
    if result['description']!="":
        # If there is a description, print a message indicating which button the user clicked
        # Use the 'description' value from 'result'
        print(f"User clicked: {result['description']}")
        # Pause for 1.5 seconds until next question and clear text
        time.sleep(1.5)
        clear_output(wait=False)
    else:
        # If there is no description, print message
        print("User did not click in time")
        # Pause for 1.5 seconds until next question and clear text
        time.sleep(1.5)
        clear_output(wait=False)
    
    # Append values into their respective key for 'results_dict' dictionary     
    results_dict['filename'].append(random_image)
    results_dict['rsponse_time'].append(response_time)
    results_dict['correct'].append(right_answer)
    results_dict['ratio'].append(rtio)

# Convert the dictionary `results_dict` into a pandas DataFrame named `resultsdataframe`
resultsdataframe = pd.DataFrame(results_dict)

# Displays final score 
print(f"You got {correct_answers} out of {questions} correct.")


# Initialize dictionary with keys corresponding to various collected data
data_dict = {
    'name': ans1,
    'gender': ans2,
    'sleep': ans3,
    'score' : correct_answers,
    # Convert `resultsdataframe` into a JSON string so it can be easily stored for google forms
    'result_json' : resultsdataframe.to_json()
}

# Check if the variable `rslt` equals the string "yes"
if rslt == "yes":
    # Submit the data contained in 'data_dict' to the google form with url 'form_url'
    send_to_google_form(data_dict, form_url)

You got 4 out of 5 correct.
